In [22]:
import pandas as pd
import numpy as np
import re
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from konlpy.tag import Okt 

In [23]:
res_lang= pd.read_csv("resume_language.csv")
langGrouped = res_lang.sort_values('resume_seq')
langGrouped.info()

<class 'pandas.core.frame.DataFrame'>
Index: 869 entries, 45 to 707
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   resume_seq  869 non-null    object 
 1   language    869 non-null    int64  
 2   exam_name   869 non-null    int64  
 3   score       869 non-null    float64
dtypes: float64(1), int64(2), object(1)
memory usage: 33.9+ KB


In [24]:
langGrouped['exam'] = langGrouped.apply(lambda x : x['language']*100+x['exam_name'], axis=1)
langGrouped.drop(['language','exam_name'], axis=1,inplace = True)                           # 언어를 100단위, 시험이름을 1단위로 설정
# 시험이름을 2~3자리 숫자로 축약

In [25]:
langGrouped['exam']=langGrouped['exam'].astype('object') # 숫자가 아닌 카테고리로 설정

In [26]:
exam_list = langGrouped.groupby('exam')['score'].apply(list) # 시험마다 평균을 확인하여 정규화하기
exam_df= pd.DataFrame(exam_list)

exam_df['mini']= 0
exam_df['maxi'] = 0
for i in range(len(exam_df['score'])):
    if len(exam_df['score'].iloc[i]) > 1:
        exam_df['mini'].iloc[i] = min(exam_df['score'].iloc[i])
        exam_df['maxi'].iloc[i] = max(exam_df['score'].iloc[i])
    else:
        exam_df['maxi'].iloc[i] = max(exam_df['score'].iloc[i])

C:\Users\user\AppData\Local\Temp\ipykernel_20204\1755443227.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exam_df['mini'].iloc[i] = min(exam_df['score'].iloc[i])
C:\Users\user\AppData\Local\Temp\ipykernel_20204\1755443227.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exam_df['maxi'].iloc[i] = max(exam_df['score'].iloc[i])
C:\Users\user\AppData\Local\Temp\ipykernel_20204\1755443227.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-

In [27]:
exam_df # 시험마다 최소, 최대치 확보, 이것을 통해서 기존 그룹의 점수를 정리하기

,score,mini,maxi
exam,,,
201,"[600.0, 848.48, 853.54, 818.18, 737.37, 848.48]",600.00,853.54
202,"[20.0, 893.94, 732.32, 30.0, 828.28, 969.7, 5....",5.56,969.70
203,[500.0],0.00,500.00
204,"[873.74, 848.48, 909.09, 5.56, 959.6, 969.7, 5...",0.00,984.85
205,"[868.69, 904.04]",868.69,904.04
207,[262.63],0.00,262.63
208,"[620.0, 590.91]",590.91,620.00
209,"[853.54, 500.0, 500.0, 762.63, 676.77, 500.0, ...",0.00,898.99
211,"[661.62, 944.44, 883.84, 848.48, 954.55, 500.0...",0.00,989.90


In [28]:
langGrouped['resu']=0
for i in range(len(langGrouped)):
    for j in range(len(exam_df)):
        if langGrouped['exam'][i] == exam_df.index[j]:
            if exam_df.iloc[j]['mini'] != exam_df.iloc[j]['maxi']:
                minn = exam_df.iloc[j]['mini']
                maxx = exam_df.iloc[j]['maxi']
                langGrouped['resu'][i] = (langGrouped['score'][i] - minn)/(maxx - minn)
            elif exam_df.iloc[i]['mini'] == exam_df.iloc[i]['maxi']:
                langGrouped['resu'][i] = langGrouped['score'][i]/1000

C:\Users\user\AppData\Local\Temp\ipykernel_20204\696078781.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  langGrouped['resu'][i] = (langGrouped['score'][i] - minn)/(maxx - minn)


In [29]:
langGrouped.drop('score',axis=1,inplace=True)
add1 = pd.get_dummies(langGrouped['exam'],prefix ='exam',dtype='int') 
add1 = add1.apply(lambda x : langGrouped['resu']*x )
langGrouped = pd.concat([langGrouped,add1],axis=1)
langGrouped = langGrouped.drop(['exam','resu'],axis=1)
langGrouped = langGrouped.set_index('resume_seq')
langGrouped

,exam_201,exam_202,exam_203,exam_204,exam_205,exam_207,exam_208,exam_209,exam_211,exam_221,...,exam_313,exam_401,exam_404,exam_405,exam_406,exam_410,exam_801,exam_901,exam_904,exam_924
resume_seq,,,,,,,,,,,,,,,,,,,,,
U00004,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.668371,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
U00005,0.0,0.000000,0.0,0.887181,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
U00039,0.0,0.000000,0.0,0.861532,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
U00054,0.0,0.000000,0.0,0.923075,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
U00067,0.0,0.014977,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
U08443,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.731952,0.0,0.0,0.0,0.0,0.0,0.0
U08461,0.0,0.000000,0.0,0.687181,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
U08465,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.011223,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
